In [1]:
from collections import Counter
import numpy as np
import os,glob

def sample_generator(filenames, wordspath, maxlen=100, minlen=10):
    def create_vocab(wpath):
        w2id = {}
        id2w = {}
        def add_word(vocab, word):
            vocab[word] = len(vocab)
        add_word(w2id, '<pad>')
        add_word(w2id, '<unk>')
        add_word(w2id, '<bos>')
        add_word(w2id, '<eos>')
        with open(wordspath) as fd:
            for line in fd:
                w, freq = line.strip().split('\t')
                add_word(w2id, w)
        for w in w2id:
            id2w[w2id[w]] = w
        return w2id, id2w
    
    def read_one_file(file, vocab, maxlen, minlen):
        with open(file) as fd:
            for line in fd:
                if not line.isspace():
                    wids = [vocab[w] if w in vocab else vocab['<unk>'] 
                           for w in list(''.join(line.strip().split()))]
                    if len(wids) < minlen or len(wids)>=maxlen:
                        continue
                    yield ([vocab['<bos>']] + wids, wids + [vocab['<eos>']], len(wids)+1)
                    
    w2id, id2w = create_vocab(wordspath)
    if isinstance(filenames, list):
        np.random.shuffle(filenames)
        for f in filenames:
            for d in read_one_file(f, w2id, maxlen, minlen):
                yield d
    else:
        for d in read_one_file(filenames, w2id, maxlen, minlen):
            yield d



In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.python.feature_column import feature_column_lib

def input_fn():
    datagen = sample_generator(glob.glob('./wiki_0000'), './words.txt')

    ds = tf.data.TextLineDataset.from_generator(lambda: datagen, (tf.int32, tf.int32, tf.int32),
                                           (tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([])))
    ds = ds.take(100)
    ds = ds.shuffle(buffer_size=100000)
    ds = ds.padded_batch(10, (tf.TensorShape([None]),
                              tf.TensorShape([None]),
                              tf.TensorShape([])))
    ds = ds.map(lambda a, b, c: ({'f_wids': a, 'f_len': c}, b))
    ds = ds.prefetch(5)
    return ds


In [3]:
import tensorflow as tf

def my_model(features, labels, mode, params):
    """DNN with three hidden layers and learning_rate=0.1."""
    in_x, in_len = features['f_wids'], features['f_len']
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': features,
            'logits': features
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

def main():

    classifier = tf.estimator.Estimator(
        model_fn=my_model, params={})
    predictions = classifier.predict(input_fn=lambda: gendata())

    print([o for o in predictions])


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.python.feature_column import feature_column_lib


aa = [o for o in range(10)]
def gen(x):
    np.random.shuffle(x)
    for i in x:
        yield (i, i)
def gendata():
    ds = tf.data.TextLineDataset.from_generator(lambda: gen(aa), (tf.int32, tf.int32),
                                                (tf.TensorShape([]), tf.TensorShape([])))
    ds = ds.batch(3).repeat(2)
    return ds

def my_model(features, labels, mode, params):
    """DNN with three hidden layers and learning_rate=0.1."""
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': features,
            'logits': features
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

def main():

    classifier = tf.estimator.Estimator(
        model_fn=my_model, params={})
    predictions = classifier.predict(input_fn=lambda: gendata())

    print([o for o in predictions])

if __name__ == '__main__':
    main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/5r/zhr6xl0j4hlgsj6383x59_g40000gn/T/tmpx9d7sq0p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb41882898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/5r/zhr6xl0j4hlgsj6383x59_g40

In [64]:
import tensorflow as tf
from tensorflow.python.client import session as tf_sesstion
# from tensorflow.python import ops
import numpy as np
aa = [o for o in range(20)]
def gen(x):
    print('shuffle')
#     np.random.shuffle(x)
    for i in x:
        yield i
def gendata():
    ds = tf.data.TextLineDataset.from_generator(lambda: gen(aa), 
                                                (tf.int32), (tf.TensorShape([]))).repeat(5)
    ds = ds.shuffle(5)
    ds = ds.batch(3).batch(4)
    return ds


g = tf.Graph()

with g.as_default():
    ds = gendata()
    ds = ds.make_one_shot_iterator()
    out = ds.get_next()
    c = out + tf.constant(1, dtype=tf.int32)
#     a = tf.constant(1.0, dtype=tf.float32)
#     b = tf.constant(2.3, dtype=tf.float32)
#     c = a + b
with tf_sesstion.Session(graph=g) as sess:
    for i in range(100):
        try: 
            print(sess.run(c))
        except tf.errors.OutOfRangeError:
            break


shuffle
[[ 5  1  6]
 [ 7  8  2]
 [ 9 12 13]
 [ 4 15 10]]
shuffle
[[17 11 18]
 [16 14  3]
 [19  3  4]
 [ 1  7  6]]
[[ 2 10  5]
 [12 20 13]
 [15 16  8]
 [11 18 20]]
shuffle
[[19  9 14]
 [ 2  4  1]
 [ 7 17  5]
 [ 3 11  6]]
shuffle
[[13  9 10]
 [14  8 16]
 [15 12 18]
 [ 1 17  2]]
[[19  4  3]
 [ 7  8  6]
 [ 5 20  9]
 [13 11 10]]
shuffle
[[12 18 17]
 [19 15 20]
 [16  2  3]
 [ 1  5  6]]
[[ 9  8 14]
 [10 12  4]
 [13 16 14]
 [17 18 11]]


InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [3] and element 1 had shape [1].
	 [[node IteratorGetNext (defined at <ipython-input-64-ea5e9c1f992d>:24) ]]

Caused by op 'IteratorGetNext', defined at:
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/Users/jerrik/a3/envs/tf1.13/lib/python3.7/site-packages/tornado/platform/asyncio.

In [20]:
import tensorflow as tf
from tensorflow.python.client import session as tf_sesstion
# from tensorflow.python import ops
import numpy as np
aa = [o for o in range(10)]
def gen(x):
    print('shuffle')
    np.random.shuffle(x)
    for i in x:
        yield i
def gendata():
    ds = tf.data.TextLineDataset.from_generator(lambda: gen(aa), 
                                                (tf.float32), (tf.TensorShape([])))
    ds = ds.batch(3).repeat(2)
    return ds
@tf.function
def run():
    ds = gendata()
    ds = ds.make_one_shot_iterator()
    ds = ds.get_next()
    c = ds + tf.constant(1, dtype=tf.float32)
    return c

g = tf.Graph()
with g.as_default():
    cc = run()+1.3


with tf_sesstion.Session(graph=g) as sess:
    for i in range(20):
        try: 
            print(sess.run(cc))
        except tf.errors.OutOfRangeError:
            break



AttributeError: module 'tensorflow' has no attribute 'function'

In [33]:
import tensorflow as tf
from tensorflow.python.client import session as tf_sesstion
# from tensorflow.python import ops
import numpy as np
aa = [o for o in range(10)]
def gen(x):
    print('shuffle')
    np.random.shuffle(x)
    for i in x:
        yield i
def gendata():
    ds = tf.data.TextLineDataset.from_generator(lambda: gen(aa), 
                                                (tf.float32), (tf.TensorShape([])))
    ds = ds.batch(3).repeat(2)
    return ds
ds = gendata()
ds = ds.make_one_shot_iterator()
@tf.function
def run():
    out = ds.get_next()
    c = out + tf.constant(1, dtype=tf.float32)
    return c
for i in range(10):
    print(run())



shuffle
tf.Tensor([7. 2. 4.], shape=(3,), dtype=float32)
tf.Tensor([ 5.  8. 10.], shape=(3,), dtype=float32)
tf.Tensor([1. 3. 6.], shape=(3,), dtype=float32)
tf.Tensor([9.], shape=(1,), dtype=float32)
shuffle
tf.Tensor([6. 2. 9.], shape=(3,), dtype=float32)
tf.Tensor([1. 7. 4.], shape=(3,), dtype=float32)
tf.Tensor([5. 8. 3.], shape=(3,), dtype=float32)
tf.Tensor([10.], shape=(1,), dtype=float32)


OutOfRangeError: End of sequence
	 [[{{node IteratorGetNextSync}}]] [Op:__inference_run_3021]

# Test Run

In [2]:
import sys
import tensorflow as tf
sys.path.append("..")
from LM.LMdataset import Dataset
ds_fn = Dataset(filepath='./', wordpath='./',batch_size=16, epoch=1)

In [3]:
config = type('', (), {})()

config.optimizer = 'adam'

config.pre_trained = False
config.partial_update_until_epoch = 0
config.vocab_sz = 4135

config.embed_size = 32
config.hidden_size = 64

config.lr = 0.0001
config.decay_steps = 1000
config.decay_rate = 0.9
config.dropout = 0.2
config.reg = 0.

In [4]:
from LM.model import model
my_model = model(config)

est_model = tf.estimator.Estimator(model_fn=my_model, params={})
est_model.train(input_fn=ds_fn)

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/5r/zhr6xl0j4hlgsj6383x59_g40000gn/T/tmp6l6qg7c4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c44e65a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
%%bash
wc words.txt

    4131    8383   27900 words.txt


In [9]:
class Solution(object):
    
    def __init__(self):
        self.arr = {}
        
    def P(self, n, l, k):
        if n == 0 or n == l:
            return 0
        if n < l:
            return float('inf')
        if (n, l, k) not in self.arr:
            return min(1+self.P(n, l+k, k), 2+self.P(n, 2*l, l)) #min(paste, copy&paste)
        else:
            return self.arr[(n, l, k)]
    
    
    def minSteps(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n <= 1:
            return 0
        return 1+ self.P(n, 1, 1)

In [19]:
import numpy as np
-np.log(1/19853)*2
print(np.exp(3.9))

49.40244910553017


In [18]:
256+128

384

In [32]:
36354/2

18177.0